## Setting up Lysoszyme to run standard MD smulation
This notebook will guide you through how to use FESetup for setting up a protein simulation and generate output for various different simulation engines. 


The notebook forms part of the CCPBio-Sim training.

*Author (2018): Antonia Mey   
Email: antonia.mey@ed.ac.uk*

*Edited (2019): Sarah Fegan*

### How to use this notebook

FESetup is a predominantly command line based tool. Executing commands in a notebook cell is possible but it is easier to work in a terminal. To open the terminal from the Jupyter home page click on New and then Terminal. To work with this notebook you will have to switch between the terminal tab and the notebook tab. It might be worthwhile separating them out into two windows and arranging them across the screen in such a way that you can look at both the terminal and the notebook. You will be required to edit some files on the command line. This can be done in `nano`, if the notebooks are run as part of the CCPBioSim online training. If you happen to be running this notebook on your computer at home feel free to use whatever editor that works for you to edit required files. 

Lines which begin with $ will be commands that should be entered into the terminal

### Imports

In [ ]:
import nglview as nv
import mdtraj as md

Change to the directory for exercise 2.

$ cd data/02_advanced

## Setting up Lysozyme 
FESetup has a nice feature, where it will produce compatible output for different MD engines. It currently supports *NAMD*, *AMBER*, *Gromacs* and *CHARMM*, provided the executables for each of the simulation packages are actually installed. In the following we will look at how to set up lysozyme.

Let's start by loading the molecule.

In [ ]:
view = nv.viewMolecules('proteins/181L/protein.pdb')
view

As for the ethane methanol simulations we have an input file for FESetup. This time it has a new section named [protein]. This contains all the relevant information for setting up a protein simulation. Let's have a look at this very basic setup file. 

$ head -n 30 setup1.in


We want to put the protein 181L into a box of length 12 add some solvent and neutralize the box and run a set of minimisation steps. Executing this will generate the ouptut file `_proteins` which will contain the minimised protein structure. 

$ FESetup setup1.in

$ ls _proteins/181L

This directory now contains your solvated protein ready for an equilibration or other type of simulation run. This is the simplest way of preparing a protein simulation. 

### Setup with Gromacs
We can use the same setup to run the minimisation with GROMACS. The only thing that will have to be changed is the line specifying the MD engine:   
`
mdengine = gromacs, mdrun
`   
If Gromacs is not installed the setup with fail. This is currently the case on this server, but we can still run an example file until we get the error. This time we use `182L` for the setup. 

$ FESetup setup_gromacs.in

While we have recieved an error, because we don't have Gromacs installed and can't run the minimisation, we can have a look in the `_proteins/182L` directory. You will find it contains a `solvated.gro` and `solvated.top` file, which are the respective coordinate and topology files associated with Gromacs and can then be used to run your simulation with gromacs.  

$ ls _proteins/182L

**Task: What would you have to change in `setup.in` to prepare files compatible with NAMD?**

## Setting up Lysozyme with a ligand 
Setting up a single protein isn't necessarily such a complicated task. But setting up a protein and a ligand together might require more effort. The ligand will often lack parameters so partial charges for the forcefield need to be generated and doing this by hand is cumbersome. FESetup can help with this. Let's look at Task02 working with both protein and ligand. 

### Benzene as a ligand of Lysozyme

You have a proteins and ligands directory. The proteins directory contains the structure of 181L and the ligands directory that of benzene.    

**Task: visualise both the protein and the ligand to make sure that the ligand has the right coordinates**

In [ ]:
#change appropriate code below
protein = md.load('path/to/protein/file')
ligand = md.load('path/to/ligand/file')
from nglview import NGLWidget
view = NGLWidget()
view.add_trajectory(protein)
view.add_trajectory(ligand)
view

Now we want to actually create a complex of the ligand and protein and solvate it. For this there is another `setup.in` file prepared. Let's look at this file a bit more closely:

$ cat setup2.in

We again have a ligand section which defines the moleucle  and also makes sure that benzene will be set up in a solvation box. Then again the same protein section, but now in addition there is the complex section, which defines the fact that we want to create a comple of 181L and benzene, but that in a box solvate it and minimise it. 

Run the setup. 

Now you should be able to located a directory called `_complex` that was generated by FESetup. It should contain the complex in vacuum and in solvent. 

$ ls _complex/181L:benzene

In [ ]:
# Insert code to visualise the complex in vacuum



**Tip: Play around with the representations in order to be able to view both the ligand and the protein**

## Ortho-xylene as a ligand of Lysozyme
Often you are in a position where you might want to run a simulation with a ligand where you only have a smiles string or sdf file. So how can we go from a smiles file to a solvated complex. One easy way to do this is using babel. 
Take a look at `ligands/ortho-xylene/ligand.smi`. This file contains the following smiles string: `CC1=C(C)C=CC=C1`. The following command creates a pdb file with a 3-D structure.

$ obabel -ismi ligands/ortho-xylene/ligand.smi  -opdb ligands/ortho-xylene/ligand.pdb --gen3D -h  > ligands/ortho-xylene/ligand.pdb

Double check that the ligand molecule has been generated!

Next there is an already prepared input file for ortho-xylene! Let's just run the setup!

$ FESetup setup_xylene.in

Let's have a look at the generated complex for ortho-xylene. There seems to be a problem with the way the ligand is positioned!

In [ ]:
view = nv.viewMolecules('_complexes/181L:ortho-xylene/vacuum.pdb')

**Question: Can you think of ways to align the ligand to the correct binding site of the protein? Write down a few ideas in the next cell:**

**Advanced task**: *Come back to this if you still have time at the end of the tutorial and implement a way to put xylene in an appropriate position in the binding site.

## Setting up ligand perturbations with a protein
If you want to compute relative binding free energies of a set of ligands to a protein using an alchemical approach you will need to generate a perturbation map. In the next task you will generate a set of ligands that are all similar and you need to define a perturbation map for them. Just think of what was discussed in the lecture in terms of perturbation maps. You want:
- many cycles, to make sure you can calculate robustness
- easy perturbations, avoid breaking large rings, if needed add intermediates
- always consider forward and backward perturbations.

We have the following ligands:
- indene,
- benzene,
- benzofurane
- indole
- o-xylene
- p-xylene
Have a look at their structures:
![foo](images/structures.png)

Tasks:
1. Change molsetup.in in such a way that all molecules are setup with FESetup and a complex is created the 181L structure without running a proper equilibration to avoid long execution times. 
2. Suggest a set of perturbations and add them to morph.in to generate a perturbation map that is sensible in order to assess relative binding free energies of the above 6 molecules. 
3. After running molsetup.in and morph.in setup look at the generated `_perturbations` directory which can then be used for running somd alchemical free energy calculations

Modify the molesetup.in before running FESetup

$ nano molsetup.in

$ FESetup molsetup.in

Modify the morph.in as needed before running FESetup

$ nano morph.in

$ FESetup morph.in

In [ ]:
# Add code to visualise some of your generated output. 




## Advanced topics: explicit mapping 
Sometimes the proposed atom mapping for the morphing is not quite what you might want to do. For this purpose FESetup has a built in feature that lets you explicitly map the atoms. This may become imporant with chiral molecules or symmetric molecules or where you think a mapping makes more sense than what is proposed by the automatic algorithm. Think of this for example:
![foo](images/mapping.png)
![foo](images/mapping2.png)
Let's see how we can achieve and explicit atom mapping. 

You can explicity state the mapping in the `setup.in` file in this way:   
```
[ligand]
basedir = ligands
morph_pairs = benzene > benzofuran /1=3/2=2  # indices start from 1
```

Or you can use a `.map` file that sits in your ligands base directory to do the mapping. 

The content of the equivalent map file would look like this:   
```
# example mapping file benzene~benzofuran.map in the basedir ligands/
# explicitly map the following atom indexes onto each other
1 3 # this mapping and...
2 2 # ...this one will fix the orientation of benzofuran in space

```

**Tasks: **
1. Run FESetup using an explicit mapping of 1=3 and 2=2 in the setup.in file.
2. Run FESetup using a map file to do the same mapping attaching to 3 and 4 of the benzene C atoms

Please modify the existing file `setup3.in` and `ligands/benzene~benzofurane.map` files to complete the task!